In [1]:
# %load simpleConvNet.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Model structure
class Net(nn.Module):
    def __init__(self):
        # 在构造函数中，实例化不同的layer组件，并赋给类成员变量
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 在前馈函数中，利用实例化的组件对网络进行搭建，并对输入Tensor进行操作，并返回Tensor类型的输出结果
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    

In [2]:
import os
if not os.path.exists('./data'):
    os.mkdir('./data')

In [3]:
# 根据环境参数选取对应运行平台：cpu or gpu
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Running Device:', device)

# 对获取的图像数据做ToTensor()变换和归一化
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 利用torchvision提供的CIFAR10数据集类，实例化训练集和测试集提取类
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 利用torch提供的DataLoader, 实例化训练集DataLoader 和 测试集DataLoader
Batch_size = 8
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=Batch_size, shuffle=True)
testLoader = torch.utils.data.DataLoader(testset, batch_size=Batch_size, shuffle=False)

# CIFAR10 类别内容
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Running Device: cpu
Files already downloaded and verified
Files already downloaded and verified


In [4]:
# 卷积神经网络实例化
net = Net().to(device)
print(net)

# 实例化损失函数和SGD优化子
epochs = 3
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 卷积神经网络训练
for epoch in range(epochs):
    running_loss = 0.0

    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data
        # 将数据迁移到device中，如device为GPU，则将数据从CPU迁移到GPU；如device为CPU，则将数据从CPU迁移到CPU（即不作移动）
        inputs, labels = inputs.to(device), labels.to(device)

        # 清空参数的梯度值
        optimizer.zero_grad()

        # 前馈+反馈+优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 统计训练损失
        running_loss += loss.item()

        if i % 2000 == 1999:    # 每2000 mini-batches，打印一次
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
[1,  2000] loss: 2.177
[1,  4000] loss: 1.788
[1,  6000] loss: 1.590
[2,  2000] loss: 1.458
[2,  4000] loss: 1.412
[2,  6000] loss: 1.338
[3,  2000] loss: 1.287
[3,  4000] loss: 1.230
[3,  6000] loss: 1.226
Finished Training


In [5]:
# 卷积神经网络测试
correct = 0
total = 0
with torch.no_grad():     # 测试阶段，停止梯度计算
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) #

        # 前馈获得网络预测结果
        outputs = net(inputs)
        # 在预测结果中，获得预测值最大化的类别id
        _, predicted = torch.max(outputs.data, 1)

        # 统计样本个数
        total += labels.size(0)
        # 统计正确预测样本个数
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test inputs: %d %%' % (100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # 前馈获得网络预测结果
        outputs = net(inputs)
        # 在预测结果中，获得预测值最大化的类别id
        _, predicted = torch.max(outputs.data, 1)

        # 对于batch内每一个样本，获取其预测是否正确
        c = (predicted == labels).squeeze()
        # 对于batch内样本，统计每个类别的预测正确率
        for i in range(Batch_size):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the 10000 test inputs: 53 %
Accuracy of plane : 69 %
Accuracy of   car : 44 %
Accuracy of  bird : 61 %
Accuracy of   cat : 11 %
Accuracy of  deer : 39 %
Accuracy of   dog : 44 %
Accuracy of  frog : 43 %
Accuracy of horse : 77 %
Accuracy of  ship : 65 %
Accuracy of truck : 77 %


In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=Batch_size, shuffle=True)

for batch in trainLoader:
    print(batch)
    X, y = batch
    print(X.shape)
    print(y.shape)
    break

Files already downloaded and verified
[tensor([[[[-0.5137, -0.5765, -0.5922,  ..., -0.5137, -0.4824, -0.5216],
          [-0.5294, -0.5373, -0.4039,  ..., -0.2784,  0.0667, -0.5137],
          [-0.5216, -0.5451, -0.4039,  ..., -0.1843,  0.1843, -0.3804],
          ...,
          [-0.0275, -0.0353, -0.0275,  ..., -0.1922, -0.1294, -0.0902],
          [-0.0039, -0.0275, -0.0980,  ..., -0.0510, -0.0431, -0.0745],
          [ 0.0118, -0.0353, -0.0980,  ..., -0.0353, -0.0431, -0.0275]],

         [[-0.4118, -0.4431, -0.4353,  ..., -0.3725, -0.3647, -0.4118],
          [-0.4196, -0.4353, -0.3176,  ..., -0.1216,  0.1529, -0.3725],
          [-0.4196, -0.4824, -0.3647,  ..., -0.1216,  0.0745, -0.3333],
          ...,
          [-0.2627, -0.2549, -0.2549,  ..., -0.3333, -0.3176, -0.2941],
          [-0.2157, -0.2235, -0.3020,  ..., -0.2627, -0.2863, -0.3333],
          [-0.1765, -0.2157, -0.2784,  ..., -0.2706, -0.2784, -0.2863]],

         [[-0.3804, -0.4275, -0.4275,  ..., -0.3725, -0.3569, -

In [7]:
# 对获取的图像数据做ToTensor()变换和归一化
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 利用torchvision提供的CIFAR10数据集类，实例化训练集和测试集提取类
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 利用torch提供的DataLoader, 实例化训练集DataLoader 和 测试集DataLoader
Batch_size = 8
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=Batch_size, shuffle=True)
testLoader = torch.utils.data.DataLoader(testset, batch_size=Batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
for i, x in enumerate([3,2,1]):
    print(i, x)

0 3
1 2
2 1


In [9]:
def f(a, b):
    print(a, b)

In [37]:
f(*[1,2])


1 2
